In [ ]:
from qem.image_fitting import ImageModelFitting
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
%matplotlib qt

file = '../data/STO/haadf.hspy'

s = hs.load(file)
image = s.data	
dx = s.axes_manager[1].scale *10 # convert nm to Angstrom
model=ImageModelFitting(image, dx=dx,units='A')

In [ ]:
model.plot() # inspect the image

In [ ]:
model.find_peaks(min_distance=10,threshold_rel=0.05) # find atomic columns by local maximum

In [ ]:
cif_file = '../data/STO/SrTiO3_mp-5229_conventional_standard.cif' 
model.map_lattice(cif_file=cif_file,elements=['Sr','Ti'], reciprocal=False,region_index=0) # map the cif unitcell to the image and tile it according the local peak positions

In [5]:
model.view_3d()

In [ ]:
model.select_atoms()

In [7]:
model.plot_coordinates(10)

In [ ]:
model.fit_background = True
params = model.init_params()
model.fit_random_batch(params, batch_size=1000, maxiter=10, tol=1e-2, plot=False, step_size=0.01,verbose=False,num_epoch=1)

In [ ]:
model.plot_fitting()

In [ ]:
plt.figure()
plt.hist(model.volume[model.atom_types==0], bins=100, alpha=0.5,label='Sr')
plt.hist(model.volume[model.atom_types==1], bins=100, alpha=0.5,label='Ti')
plt.legend()

In [54]:
model.plot_scs(per_element=True)

In [56]:
model.plot_scs_histogram()

In [ ]:
model.voronoi_integration()

In [59]:
model.plot_scs_voronoi()